In [1]:
import matplotlib.pyplot as plot
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb

from sklearn import cluster
from sklearn import model_selection
from sklearn import neural_network

/home/user/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
pd.options.display.width = 200
pd.options.display.max_columns = 50

In [3]:
train = pd.read_csv('../data/in/train.csv', delimiter=';')
test = pd.read_csv('../data/in/test.csv', delimiter=';')
y = train.cardio
train.drop('cardio', axis=1, inplace=True)

In [4]:
test['y'] = -1
test['train'] = 0

train['y'] = y.values
train['train'] = 1

In [5]:
df_all = pd.concat([train, test])
df_all['male'] = df_all.gender == 2
df_all.drop('gender', axis=1, inplace=True)
df_all.shape

(100000, 14)

In [6]:
for c in ['active', 'alco', 'smoke']:
    replacement = test.ix[test[c] != 'None', c].astype('float32').mean()
    df_all.ix[df_all[c]=='None', c] = -1
    df_all[c] = df_all[c].astype('float32')
del c, replacement

In [7]:
df_all.age /= 365

In [8]:
idx = (df_all.height == 157).values * (df_all.weight == 10).values
df_all.ix[idx, 'weight'] = 40

idx = (df_all.height == 169).values * (df_all.weight == 10).values
df_all.ix[idx, 'weight'] = 70

idx = (df_all.height == 165).values * (df_all.weight == 10).values
df_all.ix[idx, ['weight', 'ap_lo']] = 100, 110

idx = (df_all.height == 178).values * (df_all.weight == 11).values
df_all.ix[idx, 'weight'] = 110

idx = (df_all.height == 183).values * (df_all.weight == 13).values
df_all.ix[idx, 'weight'] = 130

idx = (df_all.height == 169).values * (df_all.weight == 16.3).values
df_all.ix[idx, 'weight'] = 63

idx = (df_all.height == 170).values * (df_all.weight == 20).values
df_all.ix[idx, 'weight'] = 80

idx = (df_all.height == 162).values * (df_all.weight == 21).values
df_all.ix[idx, 'weight'] = 71

idx = (df_all.height == 177).values * (df_all.weight == 22).values
df_all.ix[idx, 'weight'] = 80

idx = (df_all.height == 157).values * (df_all.weight == 23).values
df_all.ix[idx, 'weight'] = 43

idx = (df_all.height == 171).values * (df_all.weight == 29).values
df_all.ix[idx, 'weight'] = 49

idx = (df_all.weight == 200).values
df_all.ix[idx, 'weight'] = 100

idx = (df_all.height == 180).values * (df_all.weight == 183).values
df_all.ix[idx, 'weight'] = 83

idx = (df_all.height == 58).values * (df_all.weight == 183).values
df_all.ix[idx, ['height', 'weight', 'ap_lo']] = 158, 83, 100

idx = (df_all.height == 161).values * (df_all.weight == 181).values
df_all.ix[idx, ['weight', 'ap_lo']] = 121, 110

idx = (df_all.height == 250).values * (df_all.weight == 86).values
df_all.ix[idx, 'height'] = 150

idx = (df_all.height == 165).values * (df_all.weight == 180).values
df_all.ix[idx, 'weight'] = 80

idx = (df_all.height == 180).values * (df_all.weight == 180).values
df_all.ix[idx, 'weight'] = 130

idx = (df_all.weight == 180).values
df_all.ix[idx, 'weight'] = 130

idx = (df_all.weight == 178).values * (df_all.height == 80).values
df_all.ix[idx, 'height'] = 180

idx = (df_all.weight == 178).values
df_all.ix[idx, 'weight'] = 78

idx = (df_all.weight == 177).values * (df_all.height == 177).values
df_all.ix[idx, 'weight'] = 77

#--------------------1----------------------------

In [9]:
idx = (df_all.height == 162).values * (df_all.weight == 175).values
df_all.ix[idx, 'ap_lo'] = 100

idx = (df_all.weight == 175).values
df_all.ix[idx, 'weight'] = 75

idx = (df_all.height == 87).values * (df_all.weight == 173).values
df_all.ix[idx, ['height', 'weight']] = 187, 113

idx = (df_all.height == 169).values * (df_all.weight == 172).values
df_all.ix[idx, 'weight'] = 72

idx = (df_all.height == 170).values * (df_all.weight == 171).values
df_all.ix[idx, 'weight'] = 71

idx = (df_all.weight == 170).values * (df_all.male==False).values
df_all.ix[idx, 'weight'] = 70

idx = (df_all.weight == 170).values * (df_all.height == 97).values
df_all.ix[idx, 'height'] = 197

idx = (df_all.weight == 170).values
df_all.ix[idx, 'weight'] = 140

idx = (df_all.height == 165).values * (df_all.weight == 169).values
df_all.ix[idx, 'weight'] = 70

idx = (df_all.height < 100).values * (df_all.weight == 168).values
df_all.ix[idx, 'height'] += 100

idx = (df_all.weight == 168).values
df_all.ix[idx, 'weight'] -= 100

idx = (df_all.height == 104).values * (df_all.weight == 159).values
df_all.ix[idx, 'height'] = 164

idx = (df_all.height == 159).values * (df_all.weight == 165).values
df_all.ix[idx, 'weight'] -= 100

idx = (df_all.height == 159).values * (df_all.weight == 159).values
df_all.ix[idx, 'weight'] -= 100

idx = (df_all.ap_lo == 1100).values
df_all.ix[idx, 'ap_lo'] /= 10

idx = (df_all.height == 75).values
df_all.ix[idx, 'height'] += 100

idx = (df_all.height == 68).values
df_all.ix[idx, 'height'] += 100

idx = (df_all.height == 65).values
df_all.ix[idx, 'height'] += 100

idx = (df_all.ap_lo == 1000).values
df_all.ix[idx, 'ap_lo'] /= 10

idx = (df_all.height == 60).values
df_all.ix[idx, 'height'] += 100

idx = (df_all.height == 110).values
df_all.ix[idx, 'height'] = 170

idx = (df_all.height == 57).values
df_all.ix[idx, 'height'] += 100

idx = (df_all.height == 59).values
df_all.ix[idx, 'height'] += 100

idx = (df_all.height == 76).values
df_all.ix[idx, 'height'] += 100

idx = (df_all.height == 66).values
df_all.ix[idx, ['height', 'ap_hi']] = 166, 120

idx = (df_all.height == 81).values
df_all.ix[idx, 'height'] += 100

idx = (df_all.height == 55).values
df_all.ix[idx, 'height'] += 100

idx = (df_all.height == 71).values
df_all.ix[idx, 'height'] += 100

idx = (df_all.height == 74).values
df_all.ix[idx, 'height'] += 100

idx = (df_all.height == 70).values
df_all.ix[idx, 'height'] += 100

idx = (df_all.height == 52).values
df_all.ix[idx, 'height'] += 100
#--------------------2----------------------------

In [10]:
df_all.ap_hi = np.abs(df_all.ap_hi)
df_all.ap_lo = np.abs(df_all.ap_lo)

idx = (df_all.height == 50).values
df_all.ix[idx, 'height'] += 100

idx = (df_all.height == 56).values
df_all.ix[idx, 'height'] += 100

idx = (df_all.height == 72).values
df_all.ix[idx, 'height'] += 100

idx = (df_all.height == 62).values
df_all.ix[idx, 'height'] += 100

idx = (df_all.height == 64).values
df_all.ix[idx, 'height'] += 100

idx = (df_all.height == 67).values
df_all.ix[idx, 'height'] += 100

idx = (df_all.height == 91).values
df_all.ix[idx, ['height', 'weight']] += 155, 91

idx = (df_all.height == 96).values
df_all.ix[idx, 'height'] = 156

idx = (df_all.height == 98).values
df_all.ix[idx, ['height', 'ap_hi']] = 158, 120

idx = (df_all.height == 99).values
df_all.ix[idx, 'height'] = 159

idx = (df_all.height == 100).values
df_all.ix[idx, 'height'] = 160

idx = (df_all.height == 112).values
df_all.ix[idx, 'height'] = 172

idx = (df_all.height == 122).values * (df_all.weight == 161).values
df_all.ix[idx, ['height', 'weight']] = 172, 61

idx = (df_all.height == 122).values
df_all.ix[idx, 'height'] = 142

idx = (df_all.height == 102).values
df_all.ix[idx, 'height'] = 162

idx = (df_all.ap_lo == 10000).values
df_all.ix[idx, 'ap_lo'] = 100

idx = (df_all.ap_hi == 16020).values
df_all.ix[idx, 'ap_hi'] = 120

idx = (df_all.ap_hi == 14020).values
df_all.ix[idx, 'ap_hi'] = 120

idx = (df_all.ap_hi == 11500).values
df_all.ix[idx, 'ap_hi'] = 115

idx = (df_all.ap_hi == 11020).values
df_all.ix[idx, 'ap_hi'] = 120

idx = (df_all.ap_hi == 13010).values
df_all.ix[idx, 'ap_hi'] = 110

idx = (df_all.ap_hi == 12008).values
df_all.ix[idx, 'ap_hi'] = 120

idx = (df_all.ap_hi == 14900).values
df_all.ix[idx, 'ap_hi'] = 140

idx = (df_all.ap_hi == 12080).values
df_all.ix[idx, 'ap_hi'] = 120

idx = (df_all.ap_hi == 1420).values
df_all.ix[idx, 'ap_hi'] = 120

idx = (df_all.ap_hi == 1300).values
df_all.ix[idx, 'ap_hi'] = 130

idx = (df_all.ap_hi == 1500).values
df_all.ix[idx, 'ap_hi'] = 150

idx = (df_all.ap_hi == 906).values
df_all.ix[idx, ['ap_hi', 'ap_lo']] = 90, 60
#--------------------3----------------------------

In [11]:
idx = (df_all.ap_hi == 1620).values
df_all.ix[idx, 'ap_hi'] = 120

idx = (df_all.ap_lo == 8044).values
df_all.ix[idx, 'ap_lo'] = 110

idx = (df_all.ap_hi == 1400).values
df_all.ix[idx, 'ap_hi'] = 140

idx = (df_all.ap_lo == 9100).values
df_all.ix[idx, 'ap_lo'] = 100

idx = (df_all.ap_lo == 8000).values
df_all.ix[idx, 'ap_lo'] = 80

idx = (df_all.ap_lo == 8100).values
df_all.ix[idx, 'ap_lo'] = 100

idx = (df_all.ap_lo == 1200).values
df_all.ix[idx, 'ap_lo'] = 120

idx = (df_all.ap_lo == 9011).values
df_all.ix[idx, 'ap_lo'] = 110

idx = (df_all.ap_lo == 800).values
df_all.ix[idx, 'ap_lo'] = 80

idx = (df_all.ap_lo == 8500).values
df_all.ix[idx, 'ap_lo'] = 85

idx = (df_all.ap_lo == 8099).values
df_all.ix[idx, 'ap_lo'] = 100

idx = (df_all.ap_lo == 8079).values
df_all.ix[idx, 'ap_lo'] = 80

idx = (df_all.ap_lo == 1110).values
df_all.ix[idx, 'ap_lo'] = 110

idx = (df_all.ap_lo == 710).values
df_all.ix[idx, 'ap_lo'] = 70

idx = (df_all.ap_lo == 7100).values
df_all.ix[idx, 'ap_lo'] = 100

idx = (df_all.ap_lo == 2088).values
df_all.ix[idx, ['ap_hi', 'ap_lo']] = 120, 80

idx = (df_all.ap_hi == 1).values * (df_all.ap_lo > 1000).values
df_all.ix[idx, 'ap_hi'] = 110
df_all.ix[idx, 'ap_lo'] -= 1008

idx = (df_all.ap_lo == 4100).values
df_all.ix[idx, 'ap_lo'] = 100

idx = (df_all.ap_lo == 570).values
df_all.ix[idx, ['ap_hi', 'ap_lo']] = 115, 70

idx = (df_all.ap_lo == 11000).values
df_all.ix[idx, 'ap_lo'] = 110

idx = (df_all.ap_lo == 1033).values
df_all.ix[idx, 'ap_lo'] = 100

idx = (df_all.ap_lo == 4700).values
df_all.ix[idx, 'ap_lo'] = 70

idx = (df_all.ap_lo == 5700).values
df_all.ix[idx, 'ap_lo'] = 70

idx = (df_all.ap_lo == 6800).values
df_all.ix[idx, 'ap_lo'] = 80

idx = (df_all.ap_lo == 8200).values
df_all.ix[idx, 'ap_lo'] = 80

idx = (df_all.ap_lo == 9800).values
df_all.ix[idx, 'ap_lo'] = 80

idx = (df_all.ap_lo == 7099).values
df_all.ix[idx, 'ap_lo'] = 110

idx = (df_all.ap_lo > 7000).values
df_all.ix[idx, 'ap_lo'] /= 100
#--------------------4----------------------------

In [12]:
idx = (df_all.ap_lo == 1900).values
df_all.ix[idx, 'ap_lo'] = 90

idx = (df_all.ap_lo == 1001).values
df_all.ix[idx, 'ap_lo'] = 100

idx = (df_all.ap_lo == 1400).values
df_all.ix[idx, 'ap_lo'] = 140

idx = (df_all.ap_lo == 1044).values
df_all.ix[idx, 'ap_lo'] = 104

idx = (df_all.ap_lo == 1008).values
df_all.ix[idx, 'ap_lo'] = 100

idx = (df_all.ap_lo == 1022).values
df_all.ix[idx, 'ap_lo'] = 102

idx = (df_all.ap_lo == 1177).values
df_all.ix[idx, 'ap_lo'] = 117

idx = (df_all.ap_lo == 1011).values
df_all.ix[idx, 'ap_lo'] = 111

idx = (df_all.ap_lo == 1111).values
df_all.ix[idx, 'ap_lo'] = 111

idx = (df_all.ap_lo == 1120).values
df_all.ix[idx, 'ap_lo'] = 120

idx = (df_all.ap_lo == 900).values
df_all.ix[idx, 'ap_lo'] = 90

idx = (df_all.ap_lo == 170).values * (df_all.ap_hi == 200).values
df_all.ix[idx, 'ap_lo'] = 120

idx = (df_all.ap_lo == 170).values * (df_all.ap_hi == 20).values
df_all.ix[idx, ['ap_hi', 'ap_lo']] = 200, 120

idx = (df_all.ap_lo == 1130).values
df_all.ix[idx, 'ap_lo'] = 130

idx = (df_all.ap_lo == 1300).values
df_all.ix[idx, 'ap_lo'] = 130

idx = (df_all.ap_lo == 1125).values
df_all.ix[idx, 'ap_lo'] = 125

idx = (df_all.ap_lo == 1007).values
df_all.ix[idx, 'ap_lo'] = 107

idx = (df_all.ap_lo == 1066).values
df_all.ix[idx, 'ap_lo'] = 107

idx = (df_all.ap_lo == 1077).values
df_all.ix[idx, 'ap_lo'] = 107

idx = (df_all.ap_lo == 1088).values
df_all.ix[idx, 'ap_lo'] = 108

idx = (df_all.ap_lo == 1099).values
df_all.ix[idx, 'ap_lo'] = 110

idx = (df_all.ap_hi == 1502).values
df_all.ix[idx, 'ap_hi'] = 150

idx = (df_all.ap_hi == 902).values
df_all.ix[idx, 'ap_hi'] = 90

idx = (df_all.ap_hi == 1608).values
df_all.ix[idx, 'ap_hi'] = 160

idx = (df_all.ap_hi == 1130).values
df_all.ix[idx, 'ap_hi'] = 130
#--------------------4----------------------------

In [13]:
idx = (df_all.ap_hi == 907).values
df_all.ix[idx, ['ap_hi', 'ap_lo']] = 90, 70

idx = (df_all.ap_hi == 11).values * (df_all.ap_lo == 70).values
df_all.ix[idx, 'ap_hi'] = 110

idx = (df_all.ap_hi == 11).values * (df_all.ap_lo == 71).values
df_all.ix[idx, 'ap_hi'] = 110

idx = (df_all.ap_hi == 11).values * (df_all.ap_lo == 60).values
df_all.ix[idx, 'ap_hi'] = 110

idx = (df_all.ap_hi == 11).values * (df_all.ap_lo == 80).values
df_all.ix[idx, 'ap_hi'] = 110

idx = (df_all.ap_hi == 11).values * (df_all.ap_lo == 120).values
df_all.ix[idx, ['ap_hi', 'ap_lo']] = 110, 70

idx = (df_all.ap_hi == 400).values
df_all.ix[idx, 'ap_hi'] = 100

idx = (df_all.ap_hi == 10).values * (df_all.ap_lo == 160).values
df_all.ix[idx, ['ap_hi', 'ap_lo']] = 100, 60

idx = (df_all.ap_hi == 10).values
df_all.ix[idx, 'ap_hi'] = 100

idx = (df_all.ap_hi == 12).values * (df_all.ap_lo < 100).values
df_all.ix[idx, 'ap_hi'] = 120

idx = (df_all.ap_hi == 12).values
df_all.ix[idx, ['ap_hi', 'ap_lo']] = 140, 120

idx = (df_all.ap_hi == 1202).values
df_all.ix[idx, 'ap_hi'] = 120

idx = (df_all.ap_hi == 909).values
df_all.ix[idx, 'ap_hi'] = 90

idx = (df_all.ap_hi == 13).values
df_all.ix[idx, 'ap_hi'] = 130

idx = (df_all.ap_hi == 14).values
df_all.ix[idx, 'ap_hi'] = 140

idx = (df_all.ap_hi == 15).values
df_all.ix[idx, 'ap_hi'] = 150

idx = (df_all.ap_hi == 16).values * (df_all.ap_lo == 10).values
df_all.ix[idx, ['ap_hi', 'ap_lo']] = 160, 100

idx = (df_all.ap_hi == 16).values
df_all.ix[idx, 'ap_hi'] = 160

idx = (df_all.ap_hi == 957).values
df_all.ix[idx, 'ap_hi'] = 95

idx = (df_all.ap_hi == 2000).values
df_all.ix[idx, 'ap_hi'] = 200

idx = (df_all.ap_hi == 1407).values
df_all.ix[idx, 'ap_hi'] = 140

idx = (df_all.ap_hi == 1409).values
df_all.ix[idx, 'ap_hi'] = 140

idx = (df_all.ap_hi == 509).values
df_all.ix[idx, ['ap_hi', 'ap_lo']] = 150, 90

idx = (df_all.ap_hi == 17).values
df_all.ix[idx, 'ap_hi'] = 170

idx = (df_all.ap_hi == 1).values
df_all.ix[idx, ['ap_hi', 'ap_lo']] = 130, 90

idx = (df_all.ap_hi == 806).values
df_all.ix[idx, ['ap_hi', 'ap_lo']] = 80, 60

idx = (df_all.ap_lo == 850).values
df_all.ix[idx, 'ap_lo'] = 85
#--------------------5----------------------------

In [14]:
idx = (df_all.ap_hi == 1205).values
df_all.ix[idx, 'ap_hi'] = 120

idx = (df_all.ap_hi == 1110).values
df_all.ix[idx, 'ap_hi'] = 110

idx = (df_all.ap_hi == 960).values
df_all.ix[idx, 'ap_hi'] = 90

idx = (df_all.ap_hi == 701).values
df_all.ix[idx, ['ap_hi', 'ap_lo']] = 110, 70

idx = (df_all.ap_lo == 902).values
df_all.ix[idx, 'ap_lo'] = 90

idx = (df_all.ap_lo == 1003).values
df_all.ix[idx, 'ap_lo'] = 100

idx = (df_all.ap_lo == 801).values
df_all.ix[idx, 'ap_lo'] = 80

idx = (df_all.ap_lo == 1101).values
df_all.ix[idx, 'ap_lo'] = 110

idx = (df_all.ap_lo == 1002).values
df_all.ix[idx, 'ap_lo'] = 100

idx = (df_all.ap_lo == 1139).values
df_all.ix[idx, 'ap_lo'] = 140

idx = (df_all.ap_lo == 809).values
df_all.ix[idx, 'ap_lo'] = 80

idx = (df_all.weight == 165).values * (df_all.height < 190).values
df_all.ix[idx, 'weight'] = 65

idx = (df_all.ap_lo == 1004).values
df_all.ix[idx, 'ap_lo'] = 100

idx = (df_all.ap_lo == 1009).values
df_all.ix[idx, 'ap_lo'] = 110

idx = (df_all.ap_lo == 1211).values
df_all.ix[idx, 'ap_lo'] = 120

idx = (df_all.ap_lo == 910).values
df_all.ix[idx, 'ap_lo'] = 90

idx = (df_all.ap_lo == 1140).values
df_all.ix[idx, 'ap_lo'] = 140

idx = (df_all.ap_lo > 800).values
df_all.ix[idx, 'ap_lo'] /= 10

idx = (df_all.ap_hi == 160).values * (df_all.ap_lo == 708).values
df_all.ix[idx, 'ap_lo'] = 108

idx = (df_all.ap_lo > 600).values
df_all.ix[idx, 'ap_lo'] /= 10

idx = (df_all.ap_lo == 585).values
df_all.ix[idx, 'ap_lo'] = 85

idx = (df_all.ap_hi == 309).values
df_all.ix[idx, ['ap_hi', 'ap_lo']] = 130, 90

idx = (df_all.ap_lo == 0).values * (df_all.ap_hi == 108).values
df_all.ix[idx, ['ap_hi', 'ap_lo']] = 100, 80

idx = (df_all.ap_lo == 0).values * (df_all.ap_hi == 117).values
df_all.ix[idx, ['ap_hi', 'ap_lo']] = 110, 70

idx = (df_all.ap_lo == 0).values * (df_all.ap_hi == 118).values
df_all.ix[idx, ['ap_hi', 'ap_lo']] = 110, 80

idx = (df_all.ap_lo == 0).values * (df_all.ap_hi == 138).values
df_all.ix[idx, ['ap_hi', 'ap_lo']] = 130, 80

idx = (df_all.ap_lo == 0).values * (df_all.ap_hi == 148).values
df_all.ix[idx, ['ap_hi', 'ap_lo']] = 140, 80

idx = (df_all.ap_lo == 0).values * (df_all.ap_hi == 149).values
df_all.ix[idx, ['ap_hi', 'ap_lo']] = 140, 90
#--------------------6----------------------------

In [15]:
idx = (df_all.ap_hi == 90).values * (df_all.ap_lo == 160).values
df_all.ix[idx, ['ap_hi', 'ap_lo']] = 90, 60

idx = (df_all.ap_hi == 100).values * (df_all.ap_lo == 160).values
df_all.ix[idx, ['ap_hi', 'ap_lo']] = 100, 60

idx = (df_all.ap_hi == 80).values * (df_all.ap_lo == 170).values
df_all.ix[idx, ['ap_hi', 'ap_lo']] = 80, 70

idx = (df_all.ap_hi == 90).values * (df_all.ap_lo == 170).values
df_all.ix[idx, ['ap_hi', 'ap_lo']] = 90, 70

idx = (df_all.ap_hi == 95).values * (df_all.ap_lo == 180).values
df_all.ix[idx, ['ap_hi', 'ap_lo']] = 95, 80

idx = (df_all.ap_hi == 95).values * (df_all.ap_lo == 170).values
df_all.ix[idx, ['ap_hi', 'ap_lo']] = 95, 70

idx = (df_all.ap_hi == 95).values * (df_all.ap_lo == 160).values
df_all.ix[idx, ['ap_hi', 'ap_lo']] = 95, 60

idx = (df_all.ap_hi == 130).values * (df_all.ap_lo == 190).values
df_all.ix[idx, ['ap_hi', 'ap_lo']] = 130, 90

idx = (df_all.ap_hi == 140).values * (df_all.ap_lo == 190).values
df_all.ix[idx, ['ap_hi', 'ap_lo']] = 140, 90

idx = (df_all.ap_hi == 110).values * (df_all.ap_lo == 170).values
df_all.ix[idx, ['ap_hi', 'ap_lo']] = 110, 70

idx = (df_all.ap_hi == 150).values * (df_all.ap_lo == 180).values
df_all.ix[idx, ['ap_hi', 'ap_lo']] = 150, 80

idx = (df_all.ap_hi == 170).values * (df_all.ap_lo == 190).values
df_all.ix[idx, ['ap_hi', 'ap_lo']] = 170, 90

for p_h, p_l in [ (120, 80), (130, 80), (130, 90), (140, 90), (100, 70), (125, 80),
                  (110, 70), (140, 80), (150, 90), (110, 80), (120, 70), (100, 60),
                  (110, 60), (100, 80), (100, 90), (120, 90), (150, 100),(115, 80),
                  (130, 95), (150, 120),(140, 95), (130, 85), (135, 95), (105, 80),
                  (150, 95), 
                ]:
    idx = (df_all.ap_hi == p_l).values * (df_all.ap_lo == p_h).values
    df_all.ix[idx, ['ap_hi', 'ap_lo']] = p_h, p_l

idx = (df_all.ap_hi == 20).values * (df_all.ap_lo == 80).values
df_all.ix[idx, ['ap_hi', 'ap_lo']] = 120, 80

idx = (df_all.ap_hi == 20).values * (df_all.ap_lo == 90).values
df_all.ix[idx, ['ap_hi', 'ap_lo']] = 120, 90

idx = (df_all.ap_hi == 172).values * (df_all.ap_lo == 190).values
df_all.ix[idx, ['ap_hi', 'ap_lo']] = 172, 90

idx = (df_all.ap_hi == 401).values
df_all.ix[idx, 'ap_hi'] = 101

idx = (df_all.ap_hi == 7).values
df_all.ix[idx, 'ap_hi'] = 120

idx = (df_all.ap_lo == 12).values
df_all.ix[idx, 'ap_lo'] = 120

idx = (df_all.ap_lo == 10).values
df_all.ix[idx, 'ap_lo'] = 100

idx = (df_all.ap_lo < 10).values
df_all.ix[idx, 'ap_lo'] *= 10

idx = (df_all.ap_lo == 30).values
df_all.ix[idx, 'ap_lo'] = 90

idx = (df_all.ap_hi == 116).values * (df_all.ap_lo == 120).values
df_all.ix[idx, 'ap_hi'] = 146

idx = (df_all.ap_hi == 100).values * (df_all.ap_lo == 0).values
df_all.ix[idx, 'ap_lo'] = 60

idx = (df_all.ap_hi == 120).values * (df_all.ap_lo == 0).values
df_all.ix[idx, 'ap_lo'] = 80

idx = (df_all.ap_hi == 130).values * (df_all.ap_lo == 0).values
df_all.ix[idx, 'ap_lo'] = 90 - 10 * df_all.ix[idx, 'active']

idx = (df_all.ap_lo == 0).values
df_all.ix[idx, 'ap_lo'] = 90
#--------------------7----------------------------

In [16]:
idx = (df_all.ap_lo == 10).values
df_all.ix[idx, 'ap_lo'] = 100

idx = (df_all.ap_lo == 15).values
df_all.ix[idx, ['ap_hi', 'ap_lo']] = 150, 70

idx = (df_all.ap_lo == 19).values
df_all.ix[idx, 'ap_lo'] = 90

idx = (df_all.ap_lo == 20).values * (df_all.ap_hi > 99).values * (df_all.ap_hi < 131).values
df_all.ix[idx, 'ap_lo'] = 70

idx = (df_all.ap_lo == 20).values * (df_all.ap_hi == 180).values
df_all.ix[idx, 'ap_lo'] = 120

idx = (df_all.ap_lo == 20).values
df_all.ix[idx, ['ap_hi', 'ap_lo']] = 94, 70 #!!!!!!!!!!!!!!!!!!!!!!!!!!!

idx = (df_all.ap_hi == 19).values
df_all.ix[idx, 'ap_hi'] = 120

idx = (df_all.height == 246).values
df_all.ix[idx, 'height'] = 176

idx = (df_all.height == 154).values * (df_all.weight == 162).values
df_all.ix[idx, 'weight'] = 62

idx = (df_all.height == 180).values * (df_all.weight == 164).values
df_all.ix[idx, 'weight'] = 104

idx = (df_all.height == 164).values * (df_all.weight == 164).values
df_all.ix[idx, 'weight'] = 104

idx = (df_all.height == 160).values * (df_all.weight == 155).values
df_all.ix[idx, 'weight'] = 55

idx = (df_all.height == 125).values * (df_all.weight == 167).values
df_all.ix[idx, ['height', 'weight']] = 145, 107
#--------------------7----------------------------

In [17]:
#idx = (df_all.ap_lo > df_all.ap_hi).values
df_all.ix[idx]

,id,age,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,y,train,male
15366,21958,47.684932,145,107.0,180,90.0,1,2,0.0,0.0,1.0,1,1,False


In [18]:
df_all2 = pd.DataFrame()
for c in df_all.columns:
    df_all2[c] = df_all[c]
    
clf = xgb.XGBRegressor(max_depth = 5, 
                       n_estimators = 60,
                       learning_rate = 0.04,
                       subsample = 0.8,
                       #colsample_bytree = 0.8,
                       #objective = 'binary:logistic',
                       #eval_metric = 'logloss',
                       seed = 1,
                       silent = 1
                      )
clf = neural_network.MLPRegressor(hidden_layer_sizes=[125, 5])
    
df_all2['height2'] = model_selection.cross_val_predict(clf, df_all.drop('height'), df_all.height, cv=4)
df_all2['dH'] = np.abs(df_all2.height2 - df_all2.height)

df_all2['weight2'] = model_selection.cross_val_predict(clf, df_all.drop('weight'), df_all.weight, cv=4)
df_all2['dW'] = np.abs(df_all2.weight2 - df_all2.weight)

df_all2['ap_lo2'] = model_selection.cross_val_predict(clf, df_all.drop('ap_lo'), df_all.ap_lo, cv=4)
df_all2['dAL'] = np.abs(df_all2.ap_lo2 - df_all2.ap_lo)

df_all2['ap_hi2'] = model_selection.cross_val_predict(clf, df_all.drop('ap_hi'), df_all.ap_hi, cv=4)
df_all2['dAH'] = np.abs(df_all2.ap_hi2 - df_all2.ap_hi)

In [19]:
col = 'dW'
df_all2.sort_values(col, ascending=False, inplace=True)
mean, std = df_all2[col].mean(), df_all2[col].std()
idx = (df_all2[col] < mean - 3 * std).values + (df_all2[col] > mean + 3 * std).values
print(col, mean, std)
df_all2[idx]

dW 1.46534674199 1.4989721274


,id,age,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,y,train,male,height2,dH,weight2,dW,ap_lo2,dAL,ap_hi2,dAH
43998,62861,62.060274,163,70.0,200,180.0,1,1,0.0,0.0,0.0,1,1,True,171.473811,8.473811,85.409820,15.409820,158.989386,21.010614,215.829624,15.829624
38022,54282,59.643836,161,84.0,196,182.0,2,2,0.0,0.0,1.0,1,1,False,169.318593,8.318593,97.762944,13.762944,159.868094,22.131906,213.920039,17.920039
16906,24167,47.320548,170,31.0,150,90.0,2,2,0.0,0.0,1.0,1,1,True,165.195556,4.804444,42.355609,11.355609,88.591817,1.408183,149.628565,0.371435
43137,61623,60.087671,170,120.0,160,40.0,3,3,0.0,0.0,1.0,1,1,False,167.660805,2.339195,108.952246,11.047754,59.402721,19.402721,145.103365,14.896635
25021,35738,52.030137,174,158.0,140,90.0,2,2,1.0,0.0,1.0,0,1,False,177.187865,3.187865,147.311571,10.688429,89.982404,0.017596,146.978402,6.978402
27131,38785,40.098630,174,144.0,120,80.0,1,1,1.0,1.0,0.0,1,1,True,174.631817,0.631817,134.033695,9.966305,80.195852,0.195852,129.761795,9.761795
34005,48586,43.438356,163,150.0,120,80.0,1,1,0.0,0.0,1.0,1,1,False,164.619849,1.619849,140.455390,9.544610,80.039168,0.039168,131.095945,11.095945
44029,62906,56.523288,176,68.0,210,150.0,1,1,0.0,0.0,1.0,1,1,True,180.005337,4.005337,77.438258,9.438258,140.765227,9.234773,211.715128,1.715128
31811,45437,57.887671,163,73.0,180,140.0,3,1,1.0,0.0,1.0,1,1,True,168.184765,5.184765,82.080169,9.080169,128.961187,11.038813,187.225151,7.225151
18559,26503,49.698630,160,30.0,120,80.0,1,1,0.0,0.0,1.0,1,1,False,154.574942,5.425058,38.806287,8.806287,77.740667,2.259333,120.445197,0.445197
